<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./data/total_data.csv')
print(data.shape)
data.head(3)

(2056899, 17)


,계절,월,일,요일,공휴일,성별,연령대,대분류명,소분류명,평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,체감온도(℃),일 미세먼지 농도(㎍/㎥),10만건당 건수,구매건수
0,겨울,1,1,0,1.0,F,20,식품,가공란,0.0,6.84,51.0,8.7,-2.810026,32.962963,0.480964,37
1,겨울,1,1,0,1.0,F,30,식품,가공란,0.0,6.84,51.0,8.7,-2.810026,32.962963,0.480964,16
2,겨울,1,1,0,1.0,F,40,식품,가공란,0.0,6.84,51.0,8.7,-2.810026,32.962963,0.480964,9


In [15]:
# 뷰티 데이터만 추출
data2 = data[data['대분류명']=='뷰티'].reset_index(drop=True)
print(data2.shape)
data2.head(3)

(697853, 17)


,계절,월,일,요일,공휴일,성별,연령대,대분류명,소분류명,평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,체감온도(℃),일 미세먼지 농도(㎍/㎥),10만건당 건수,구매건수
0,겨울,1,1,0,1.0,F,20,뷰티,기능성 링클케어 화장품,0.0,6.84,51.0,8.7,-2.810026,32.962963,12.154295,2
1,겨울,1,1,0,1.0,F,40,뷰티,기능성 링클케어 화장품,0.0,6.84,51.0,8.7,-2.810026,32.962963,12.154295,1
2,겨울,1,1,0,1.0,F,20,뷰티,기능성 모공관리 화장품,0.0,6.84,51.0,8.7,-2.810026,32.962963,36.000828,7


In [16]:
# 결측치 확인 및 제거
data2.isnull().sum()

계절                   0
월                    0
일                    0
요일                   0
공휴일                  0
성별                   0
연령대                  0
대분류명                 0
소분류명                 0
평균일강수량(mm)           0
평균풍속(km/h)           0
평균습도(%rh)            0
일조합                  0
체감온도(℃)              0
일 미세먼지 농도(㎍/㎥)    1884
10만건당 건수             0
구매건수                 0
dtype: int64

In [17]:
data2.dropna(inplace=True)
data2.reset_index(drop=True, inplace=True)
print(data2.shape)
data2.head(3)

(695969, 17)


,계절,월,일,요일,공휴일,성별,연령대,대분류명,소분류명,평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,체감온도(℃),일 미세먼지 농도(㎍/㎥),10만건당 건수,구매건수
0,겨울,1,1,0,1.0,F,20,뷰티,기능성 링클케어 화장품,0.0,6.84,51.0,8.7,-2.810026,32.962963,12.154295,2
1,겨울,1,1,0,1.0,F,40,뷰티,기능성 링클케어 화장품,0.0,6.84,51.0,8.7,-2.810026,32.962963,12.154295,1
2,겨울,1,1,0,1.0,F,20,뷰티,기능성 모공관리 화장품,0.0,6.84,51.0,8.7,-2.810026,32.962963,36.000828,7


In [28]:
# 데이터 용량이 큰 관계로 계절별로 나누어 모델링 적용
봄 = data2[data2['계절']=='봄'].reset_index(drop=True)
여름 = data2[data2['계절']=='여름'].reset_index(drop=True)
가을 = data2[data2['계절']=='가을'].reset_index(drop=True)
겨울 = data2[data2['계절']=='겨울'].reset_index(drop=True)

print(봄.shape)
print(여름.shape)
print(가을.shape)
print(겨울.shape)

(178079, 17)
(177458, 17)
(169428, 17)
(171004, 17)


In [29]:
봄.head(3)

,계절,월,일,요일,공휴일,성별,연령대,대분류명,소분류명,평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,체감온도(℃),일 미세먼지 농도(㎍/㎥),10만건당 건수,구매건수
0,봄,3,1,3,1.0,F,20,뷰티,화장 퍼프,0.9,16.56,58.0,8.8,-1.216191,33.592593,21.806179,50
1,봄,3,1,3,1.0,F,30,뷰티,화장 퍼프,0.9,16.56,58.0,8.8,-1.216191,33.592593,21.806179,39
2,봄,3,1,3,1.0,F,40,뷰티,화장 퍼프,0.9,16.56,58.0,8.8,-1.216191,33.592593,21.806179,20


In [30]:
# 봄 데이터 이상치 확인
def outliers(data):
    q1, q3 = np.percentile(data, [25,75])
    iqr = q3 - q1
    upper_bound = q3 + (iqr * 1.5)
    lower_bound = q1 - (iqr * 1.5)
    return np.where((data > upper_bound) | (data < lower_bound))

In [31]:
강수량_idx = outliers(봄['평균일강수량(mm)'])[0]
len(강수량_idx)

34801

In [32]:
소셜_idx = outliers(봄['10만건당 건수'])[0]
len(소셜_idx)

18586

In [33]:
미세먼지_idx = outliers(봄['일 미세먼지 농도(㎍/㎥)'])[0]
len(미세먼지_idx)

7724

In [34]:
풍속_idx = outliers(봄['평균풍속(km/h)'])[0]
len(풍속_idx)

5767

In [35]:
# 이상치 평균으로 처리
강수량_mean = 봄['평균일강수량(mm)'].mean()
봄.loc[강수량_idx, '평균일강수량(mm)'] = 강수량_mean

소셜_mean = 봄['10만건당 건수'].mean()
봄.loc[소셜_idx, '10만건당 건수'] = 소셜_mean

미세먼지_mean = 봄['일 미세먼지 농도(㎍/㎥)'].mean()
봄.loc[미세먼지_idx, '일 미세먼지 농도(㎍/㎥)'] = 미세먼지_mean

풍속_mean = 봄['평균풍속(km/h)'].mean()
봄.loc[풍속_idx, '평균풍속(km/h)'] = 풍속_mean

print(봄.shape)
봄.head(3)

(178079, 17)


,계절,월,일,요일,공휴일,성별,연령대,대분류명,소분류명,평균일강수량(mm),평균풍속(km/h),평균습도(%rh),일조합,체감온도(℃),일 미세먼지 농도(㎍/㎥),10만건당 건수,구매건수
0,봄,3,1,3,1.0,F,20,뷰티,화장 퍼프,0.9,7.619634,58.0,8.8,-1.216191,33.592593,21.806179,50
1,봄,3,1,3,1.0,F,30,뷰티,화장 퍼프,0.9,7.619634,58.0,8.8,-1.216191,33.592593,21.806179,39
2,봄,3,1,3,1.0,F,40,뷰티,화장 퍼프,0.9,7.619634,58.0,8.8,-1.216191,33.592593,21.806179,20


In [36]:
from pycaret.regression import *

reg = setup(data=봄, target='구매건수', train_size=0.8, categorical_features='일')

,Description,Value
0,session_id,859
1,Target,구매건수
2,Original Data,"(178079, 17)"
3,Missing Values,False
4,Numeric Features,7
5,Categorical Features,9
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(142463, 188)"


In [ ]:
compare_models(fold=3, sort='RMSLE')

IntProgress(value=0, description='Processing: ', max=94)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:19:16
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Huber Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,18.1030,1329.9801,36.4687,0.4669,1.1569,3.8957,2.0967
br,Bayesian Ridge,18.1013,1329.9653,36.4685,0.4669,1.1572,3.8977,2.1367
ridge,Ridge Regression,18.1083,1329.9603,36.4685,0.4669,1.1575,3.9008,1.1600
omp,Orthogonal Matching Pursuit,19.7609,1481.0358,38.4841,0.4063,1.2224,4.1296,0.3300
par,Passive Aggressive Regressor,21.4581,1536.0741,39.1390,0.3841,1.3252,5.1416,0.8567
lasso,Lasso Regression,21.4164,2003.6708,44.7619,0.1969,1.3341,4.4550,1.4100
en,Elastic Net,23.0436,2347.0836,48.4464,0.0592,1.4375,5.1549,0.3300
llar,Lasso Least Angle Regression,24.4616,2494.8170,49.9478,-0.0000,1.5379,5.8784,0.3267
lar,Least Angle Regression,495288973.2999,1098319786852251520.0000,606734993.8481,-444216894174578.4375,13.4712,158149099.6468,0.3167
